In [ ]:
"""
全世界指數比較
1. 取得全世界指數
2. 計算指數報酬、風險
3. 繪製在點圖上 X軸為風險 Y軸為報酬
"""

# 載入函數工具檔
import requests
import pandas as pd
import matplotlib.pyplot as plt
# 以下為自訂函數「getDataYF」使用
import os
import yfinance as yf

# 繪圖套件載入中文字體
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False


def getDataYF(prod):
    # 備份文件的資料夾
    folder_name = "data"
    # 檢查資料夾是否存在
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # 1. 定義備份檔案名稱
    bakfile = os.path.join(folder_name, f"yf_{prod}.csv")
    # 2. 檢視是否有歷史資料(本地端)
    if os.path.exists(bakfile):
        yfdata = pd.read_csv(bakfile)
        yfdata["Date"] = pd.to_datetime(yfdata["Date"], format="%Y-%m-%d")
        yfdata.set_index("Date", inplace=True)
    else:
        # 最大範圍
        yfdata = yf.download(prod, period="max")
        # 也可以指定起迄日期
        # yfdata = yf.download(str(prod), start='2020-01-01', end='2024-03-31')
        yfdata.columns = [i.lower() for i in yfdata.columns]
        # 上網下載後 存到本地端
        yfdata.to_csv(bakfile)
    return yfdata


# 取得多商品歷史報酬率
def getMultipleReturn(getDataFunction, symbols, price_column):
    # 透過迴圈將不同商品報酬率計算出來
    datas = []
    for symbol in symbols:
        data = getDataFunction(symbol)
        # 分別計算報酬率
        returns = data[price_column].pct_change()
        cap_ret = 1 + returns
        cap_ret.name = symbol
        datas.append(cap_ret)
    # 將多商品報酬率整合
    all_ret = pd.concat(datas, axis=1)

    return all_ret


# 取出全世界指數
url = "https://finance.yahoo.com/world-indices"
response = requests.get(url)
yahoo_webpage = pd.read_html(response.text)
world_index_page = yahoo_webpage[0]
world_index_symbols = world_index_page["Symbol"].to_list()

# 抓到所有指數的歷史報酬率
ret_dataframe = getMultipleReturn(getDataYF, world_index_symbols, "adj close")

# 計算出每一個指數的年化報酬以及年化風險
return_risk = []
for col in ret_dataframe.columns:
    symbol_name = world_index_page.loc[
        world_index_page["Symbol"] == col, "Name"
    ].iloc[0]
    returns = ret_dataframe[col]
    ann_ret = (returns).prod() ** (252 / returns.count()) - 1
    ann_risk = (returns - 1).std() * (252**0.5)
    ann_sharp = ann_ret / ann_risk
    return_risk.append([symbol_name, ann_ret, ann_risk, ann_sharp])
# 每檔商品的風險報酬
rsdf = pd.DataFrame(return_risk)
rsdf.columns = ["名稱", "年化報酬", "年化風險", "夏普比率"]
rsdf.set_index("名稱", inplace=True)
rsdf = rsdf[rsdf["夏普比率"] < 0.5]
rsdf = rsdf[rsdf["年化風險"] < 1]

# 繪製scatter 點圖
ax = rsdf.plot.scatter(x="年化風險", y="年化報酬", c="夏普比率", colormap="viridis")

# 將商品代碼文字加上去圖表
for i, txt in enumerate(rsdf.transpose()):
    ax.annotate(txt, (rsdf["年化風險"].iloc[i] + 0.001, rsdf["年化報酬"].iloc[i]))